In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv
/kaggle/input/qm9-local/qm9.json


In [6]:
# !pip install duckdb

In [8]:
# !pip install molSimplify

QML might ask for a numpy downgrade. Uncomment the following line and run if that's the case. But at first attempt, try to keep this commented and run the other cells and check if qml import works okay.

In [ ]:
# !pip install numpy==1.22.4 

Uncomment the following lines and run to install QML and dependencies.

In [9]:
# !conda install -y -c intel mkl mkl-devel mkl-static mkl-include
# !apt-get install -y gfortran
# %env MKLROOT=/opt/conda/lib
# !pip3 install git+https://github.com/qmlcode/qml@develop --user -U

FCHLs are kernel-based molecular representations. So let's first understand what do kernels mean. A good source is the notebook by Logan Ward here, which we'll follow in the cells below. 
https://github.com/WardLT/applied-ai-for-materials/blob/main/molecular-property-prediction/kernel-methods/fchl-in-one-notebook.ipynb

Although, be careful, the meaning of a kernel can be confusing if this is your first time trying to understand what kernels are. A Kernel matrix K(xi,xj) provides us the dot product between two vectors, xi and xj (these are in the input feature space) when mapped into another feature space. The new feature space is known as the feature map of the given kernel.

In [10]:
%matplotlib inline
from matplotlib import pyplot as plt
from qml.fchl import get_local_kernels, get_local_symmetric_kernels
from qml import Compound
from sklearn.model_selection import GridSearchCV, ShuffleSplit, train_test_split
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics import mean_absolute_error
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from scipy.stats import linregress
from time import perf_counter
from io import StringIO
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import duckdb
# from rdkit import Chem
# from rdkit.Chem import AllChem
from molSimplify.Classes.mol3D import mol3D

2024-06-07 13:00:59.141621: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 13:00:59.141895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 13:00:59.341228: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 1000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 1000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [13]:
def make_xyz(mol_smiles):
    mol_3d = mol3D()
    mol_3d.read_smiles(mol_smiles)
    return mol_3d.writexyz('', writestring=True)

In [14]:
df['molecule'] = df['molecule_smiles'].apply(make_xyz)

FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup

*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 2 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 19 17


FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed


*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 2 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 19 17


FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup failed
FF setup

In [15]:
type(df['molecule'])

pandas.core.series.Series

In [16]:
def extract_num_atoms(molecule):
    # Split the molecule string by new line and get the first line
    first_line = molecule.split('\n')[0]
    # Convert the first line to an integer (number of atoms)
    return int(first_line)

In [17]:
num_atoms = df['molecule'].apply(extract_num_atoms)

In [18]:
num_atoms.max()

104

In [27]:
def make_representation(x):
    # Step 1: Compute the representation
    c = Compound(StringIO(x))
    c.generate_fchl_representation(max_size=104)
    
    # Step 2: Return it
    return c.representation

In [30]:
mol0_fchl_rep = make_representation(df['molecule'][0])

In [41]:
mol0_fchl_rep

array([[[ 0.00000000e+000,  1.06999949e+000,  1.07000004e+000, ...,
          1.00000000e+100,  1.00000000e+100,  1.00000000e+100],
        [ 6.00000000e+000,  1.00000000e+000,  1.00000000e+000, ...,
          0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
        [ 0.00000000e+000, -1.58753000e-001, -7.03702000e-001, ...,
          0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
        [ 0.00000000e+000, -6.13949000e-001,  8.06040000e-001, ...,
          0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
        [ 0.00000000e+000, -8.61837000e-001, -1.76000000e-003, ...,
          0.00000000e+000,  0.00000000e+000,  0.00000000e+000]],

       [[ 0.00000000e+000,  1.31557630e+000,  1.41335558e+000, ...,
          1.00000000e+100,  1.00000000e+100,  1.00000000e+100],
        [ 6.00000000e+000,  7.00000000e+000,  6.00000000e+000, ...,
          0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
        [ 0.00000000e+000,  9.99000000e-001,  2.14000000e-001, ...,
      

In [31]:
mol0_fchl_rep.shape

(104, 5, 104)

In [32]:
mol1_fchl_rep = make_representation(df['molecule'][1])

In [42]:
mol1_fchl_rep.shape

(104, 5, 104)

In [38]:
mol2_fchl_rep = make_representation(df['molecule'][2])

In [43]:
all_fchls = [mol0_fchl_rep, mol1_fchl_rep, mol2_fchl_rep]

In [ ]:
df['fchl_rep'] = df['molecule'].apply(make_representation)

In [34]:
class FCHLKernel(BaseEstimator):
    """Class for computing the kernel matrix using the qml utility functions
    
    The input `X` to all of the function is a list of FCHL representation vectors
    
    Follows the "BaseEstimator" API so that we can 
    """
    
    def __init__(self):
        super(FCHLKernel, self).__init__()
        self.train_points = None
    
    def fit(self, X, y=None):
        # Store the training set
        self.train_points = np.array(X)
        return self
        
    def fit_transform(self, X, y=None):
        self.fit(X)
        # Uses the get_localget_local_symmetric_kernels to halve the
        #  computational cost (as the matrix is symmetric)
        return np.squeeze(get_local_symmetric_kernels(self.train_points))
    
    def transform(self, X, y=None):
        return get_local_kernels(np.array(X), self.train_points)[0]

In [35]:
fchl_kernel = FCHLKernel()

In [ ]:
fchl_kernel.fit_transform(df['fchl_rep'].tolist())

In [44]:
fchl_kernel.fit_transform(all_fchls)

array([[3583.10645376, 3490.48961456, 4001.39485034],
       [3490.48961456, 3410.12806022, 3910.75350571],
       [4001.39485034, 3910.75350571, 4487.26421774]])

In [47]:
fchl_kernel.transform([mol2_fchl_rep])

array([[4001.39485034, 3910.75350571, 4487.26421774]])